# Scale factor vs time using astropy

- author : Sylvie Dagoret-Campagne
- creation date : 2025-07-08
- kernel : my-manim-environnement (with astropy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18 as cosmo  # Ou un autre modèle cosmologique
from matplotlib.ticker import FuncFormatter
import astropy.units as u
from scipy.interpolate import interp1d

In [ ]:

# Redshift allant de z = 0 à z = 1000
z = np.logspace(-3, 3, 1000)  # Échelle logarithmique pour mieux voir les débuts
a = 1 / (1 + z)

# Temps cosmique en Gyr (converti depuis lookback_time)
t = cosmo.age(z).value  # âge de l'univers à chaque redshift en Gyr

# Affichage
fig, ax1 = plt.subplots(figsize=(10, 6))

# Courbe principale : a(t)
ax1.plot(t[::-1], a[::-1], color='blue')  # On inverse pour que le temps croisse
ax1.set_xlabel("Temps depuis le Big Bang (Gyr)")
ax1.set_ylabel("Facteur d’échelle a(t)")
ax1.set_title("Facteur d’échelle en fonction du temps cosmique")
ax1.grid(True)

# Axe secondaire : redshift z(t)
def time_to_redshift(t_Gyr):
    from scipy.interpolate import interp1d
    return interp1d(t[::-1], z[::-1], kind='linear', bounds_error=False, fill_value="extrapolate")(t_Gyr)

# Ajout de l’axe en haut (redshift)
ax2 = ax1.twiny()
tick_times = np.linspace(t.min(), t.max(), 8)
tick_redshifts = time_to_redshift(tick_times)
tick_labels = [f"{zi:.1f}" if zi < 10 else f"{zi:.0f}" for zi in tick_redshifts]
ax2.set_xticks(tick_times)
ax2.set_xticklabels(tick_labels)
ax2.set_xlabel("Redshift z")

plt.tight_layout()
plt.show()


In [ ]:
# Redshift de 0 (aujourd'hui) à très grand (Big Bang)
z = np.logspace(-3, 4, 1000)  # jusqu'à z=10 000 pour mieux explorer le passé
a = 1 / (1 + z)
t = cosmo.age(z).value  # âge de l’univers en Gyr

# Supprime les points où t=0 (infiniment proche du Big Bang, log(0) impossible)
mask = t > 0
t = t[mask]
a = a[mask]
z = z[mask]

# Affichage avec échelle log en temps
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.plot(t, a, color='blue')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel("Temps depuis le Big Bang (Gyr) [échelle log]")
ax1.set_ylabel("Facteur d’échelle a(t)")
ax1.set_title("Facteur d’échelle vs temps cosmique (log)")
ax1.grid(True, which='both', ls='--')

# Axe secondaire : redshift
from scipy.interpolate import interp1d
time_to_z = interp1d(t, z, bounds_error=False, fill_value="extrapolate")

ax2 = ax1.twiny()
tick_times = np.logspace(np.log10(t.min()), np.log10(t.max()), 8)
tick_redshifts = time_to_z(tick_times)
tick_labels = [f"{zi:.1f}" if zi < 10 else f"{zi:.0f}" for zi in tick_redshifts]

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xticks(tick_times)
ax2.set_xticklabels(tick_labels)
ax2.set_xlabel("Redshift z")

plt.tight_layout()
plt.show()

In [ ]:
# Échantillonnage des redshifts : de l'époque moderne jusqu'au Big Bang
z = np.logspace(-3, 4, 1000)
a = 1 / (1 + z)
t = cosmo.age(z).value  # âge de l’univers en Gyr

# Supprimer les points où t <= 0 (log(0) impossible)
mask = t > 0
t = t[mask]
a = a[mask]
z = z[mask]

# Préparer interpolation pour les annotations
z_of_t = interp1d(t, z, bounds_error=False, fill_value="extrapolate")
a_of_t = interp1d(t, a, bounds_error=False, fill_value="extrapolate")

# Dictionnaire d’événements clés : nom -> temps en Gyr estimé
events = {
    "Inflation": 1e-32,           # très spéculatif, mais montrable sur un axe log
    "Recombinaison (CMB)": 0.00038,  # ~380,000 ans
    "Réionisation": 0.5,          # entre z ~ 6 et 10
    "Formation de la Voie Lactée": 1.0,
    "Aujourd'hui": cosmo.age(0).value,
}

# Création du graphique
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.plot(t, a, color='blue', lw=2)
ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel("Temps depuis le Big Bang (Gyr) [log]")
ax1.set_ylabel("Facteur d’échelle a(t) [log]")
ax1.set_title("Évolution du facteur d’échelle dans l’univers (log-log)")
ax1.grid(True, which='both', ls='--', alpha=0.5)

# Ajout des annotations
for name, t_event in events.items():
    if t_event > t.min():
        a_event = a_of_t(t_event)
        ax1.plot(t_event, a_event, 'ro')
        ax1.annotate(
            name,
            xy=(t_event, a_event),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=9,
            arrowprops=dict(arrowstyle='->', lw=0.5)
        )

# Axe secondaire : redshift z(t)
ax2 = ax1.twiny()
time_to_z = interp1d(t, z, bounds_error=False, fill_value="extrapolate")
tick_times = np.logspace(np.log10(t.min()), np.log10(t.max()), 8)
tick_redshifts = time_to_z(tick_times)
tick_labels = [f"{zi:.1f}" if zi < 10 else f"{zi:.0f}" for zi in tick_redshifts]

ax2.set_xscale('log')
ax2.set_xticks(tick_times)
ax2.set_xticklabels(tick_labels)
ax2.set_xlabel("Redshift z")

plt.tight_layout()
plt.show()

In [ ]:
def T_of_t(t_gyr):
    t_sec = t_gyr * 3.1536e16
    return 1e10 * (1 / t_sec)**0.5

def t_of_T(T):
    t_sec = (1e10 / T)**2
    return t_sec / 3.1536e16

In [ ]:
# Phase d’inflation
t_pl = 3.17e-38
t_inf_end = 1e-8

N = 60
t_inf = np.logspace(np.log10(t_pl), np.log10(t_inf_end), 500)
H_inf = N / (t_inf[-1] - t_inf[0])
a_pl = 1e-50
a_inf = a_pl * np.exp(H_inf * (t_inf - t_inf[0]))

# Phase standard (astropy)
z_std = np.logspace(8, -3, 1000)
z_std = np.append(z_std,0.)

a_std = 1 / (1 + z_std)
t_std = cosmo.age(z_std).value

mask = t_std > t_inf_end
t_std = t_std[mask]
a_std = a_std[mask]
z_std = z_std[mask]  # ← AJOUTE cette ligne pour corriger


# Fix pour continuité à la jonction inflation → standard
a_std *= a_inf[-1] / a_std[0]


# Fusion
t_all = np.concatenate([t_inf, t_std])
a_all = np.concatenate([a_inf, a_std])

# normalize scale factor to today
a_all = a_all/a_all[-1]

# Interpolation
z_of_t = interp1d(t_std, z_std, bounds_error=False, fill_value="extrapolate")
a_of_t = interp1d(t_all, a_all, bounds_error=False, fill_value="extrapolate")

# Événements
events = {
    "Échelle de Planck": t_pl,
    #"Inflation": t_inf[1],
    "Début inflation": 1e-10,
    "Fin inflation": t_inf[-1],
    "Recombinaison (CMB)": 0.00038,
    "Réionisation": 0.5,
    "Formation Voie Lactée": 1.0,
    "Aujourd’hui": cosmo.age(0).value,
}

# Création graphique
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(t_all, a_all, lw=2, color='darkblue', label="a(t)")
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel("Temps depuis le Big Bang (Gyr) [log]")
ax1.set_ylabel("Facteur d’échelle a(t) [log]")
ax1.set_title("Évolution du facteur d’échelle avec inflation et phases colorées")
ax1.grid(True, which='both', ls='--', alpha=0.5)


#ax1.set_xlabel("Temps [Gyr]", labelpad=10)
#ax1.xaxis.set_label_position('top')
#ax1.xaxis.tick_top()



# Colorier les phases
phases = [
    (t_all[0], t_inf_end, 'plum', "Inflation"),
    (t_inf_end, 0.05, 'orange', "Domination rad."),
    (0.05, 9, 'lightgreen', "Domination matière"),
    (9, t_all[-1], 'lightblue', "Énergie noire"),
]

for tmin, tmax, color, label in phases:
    ax1.axvspan(tmin, tmax, color=color, alpha=0.3, label=label)

# Annotations des événements
for name, t_event in events.items():
    #print(f"name = {name}, t_event = {t_event}, t_event_max = {t_all[-1]}, a= {a_event}")
    if t_event >= t_all[0] and t_event <= t_all[-1]:
        a_event = a_of_t(t_event)
        ax1.plot(t_event, a_event, 'ro')
        ax1.annotate(name, xy=(t_event, a_event), xytext=(5, 5), textcoords='offset points',
                     fontsize=8, arrowprops=dict(arrowstyle='->', lw=0.5))
       

# Axe secondaire : redshift
ax2 = ax1.twiny()
tick_times = np.logspace(np.log10(t_std.min()), np.log10(t_std.max()), 8)
tick_redshifts = z_of_t(tick_times)
tick_labels = [f"{zi:.1f}" if zi < 10 else f"{zi:.0f}" for zi in tick_redshifts]

ax2.set_xscale('log')
ax2.set_xticks(tick_times)
ax2.set_xticklabels(tick_labels)
ax2.set_xlabel("Redshift z")


# Légende et affichage
ax1.legend(loc='lower right', fontsize=8)

fig.subplots_adjust(top=0.85, bottom=0.2)

plt.tight_layout()
plt.show()


In [ ]:
# Phase d’inflation
#t_pl = 3.17e-38
t_pl = 1e-11
t_inf_end = 1e-8

N = 60
t_inf = np.logspace(np.log10(t_pl), np.log10(t_inf_end), 500)
H_inf = N / (t_inf[-1] - t_inf[0])
a_pl = 1e-50
a_inf = a_pl * np.exp(H_inf * (t_inf - t_inf[0]))

# Phase standard (astropy)
z_std = np.logspace(8, -3, 1000)
z_std = np.append(z_std,0.)

a_std = 1 / (1 + z_std)
t_std = cosmo.age(z_std).value

mask = t_std > t_inf_end
t_std = t_std[mask]
a_std = a_std[mask]
z_std = z_std[mask]  # ← AJOUTE cette ligne pour corriger


# Fix pour continuité à la jonction inflation → standard
a_std *= a_inf[-1] / a_std[0]


# Fusion
t_all = np.concatenate([t_inf, t_std])
a_all = np.concatenate([a_inf, a_std])

# normalize scale factor to today
a_all = a_all/a_all[-1]


# Interpolation
z_of_t = interp1d(t_std, z_std, bounds_error=False, fill_value="extrapolate")
a_of_t = interp1d(t_all, a_all, bounds_error=False, fill_value="extrapolate")

# Événements
events = {
    #"Échelle de Planck": t_pl,
    #"Inflation": t_inf[1],
    "Début inflation": 1e-10,
    "Fin inflation": t_inf[-1],
    "Recombinaison (CMB)": 0.00038,
    "Réionisation": 0.5,
    "Formation Voie Lactée": 1.0,
    "Aujourd’hui": cosmo.age(0).value,
}

# Création graphique
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(t_all, a_all, lw=2, color='darkblue', label="a(t)")
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel("Temps depuis le Big Bang (Gyr) [log]")
ax1.set_ylabel("Facteur d’échelle a(t) [log]")
ax1.set_title("Évolution du facteur d’échelle avec inflation et phases colorées")
ax1.grid(True, which='both', ls='--', alpha=0.5)


#ax1.set_xlabel("Temps [Gyr]", labelpad=10)
#ax1.xaxis.set_label_position('top')
#ax1.xaxis.tick_top()



# Colorier les phases
phases = [
    (t_all[0], t_inf_end, 'plum', "Inflation"),
    (t_inf_end, 0.05, 'orange', "Domination rad."),
    (0.05, 9, 'lightgreen', "Domination matière"),
    (9, t_all[-1], 'lightblue', "Énergie noire"),
]

for tmin, tmax, color, label in phases:
    ax1.axvspan(tmin, tmax, color=color, alpha=0.3, label=label)

# Annotations des événements
for name, t_event in events.items():
    #print(f"name = {name}, t_event = {t_event}, t_event_max = {t_all[-1]}, a= {a_event}")
    if t_event >= t_all[0] and t_event <= t_all[-1]:
        a_event = a_of_t(t_event)
        ax1.plot(t_event, a_event, 'ro')
        ax1.annotate(name, xy=(t_event, a_event), xytext=(5, 5), textcoords='offset points',
                     fontsize=8, arrowprops=dict(arrowstyle='->', lw=0.5))
       

# Axe secondaire : redshift
ax2 = ax1.twiny()
tick_times = np.logspace(np.log10(t_std.min()), np.log10(t_std.max()), 8)
tick_redshifts = z_of_t(tick_times)
tick_labels = [f"{zi:.1f}" if zi < 10 else f"{zi:.0f}" for zi in tick_redshifts]

ax2.set_xscale('log')
ax2.set_xticks(tick_times)
ax2.set_xticklabels(tick_labels)
ax2.set_xlabel("Redshift z")


# Légende et affichage
ax1.legend(loc='lower right', fontsize=8)

fig.subplots_adjust(top=0.85, bottom=0.2)

plt.tight_layout()
plt.show()
